In [146]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import calendar
import contextily as ctx
import glob
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [170]:
#read and get the taxi zone
taxi_zones = pd.read_csv("Summer Project/taxi_zones.csv")
manhattan_zones = taxi_zones[taxi_zones['borough'] == 'Manhattan']

In [171]:
manhattan_zones

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
9,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan
12,12,0.036661,MULTIPOLYGON (((-74.01565756599994 40.70483308...,0.000042,Battery Park,12,Manhattan
13,13,0.050281,MULTIPOLYGON (((-74.01244109299991 40.71905767...,0.000149,Battery Park City,13,Manhattan
40,41,0.052793,MULTIPOLYGON (((-73.94773985499985 40.80959972...,0.000143,Central Harlem,41,Manhattan
...,...,...,...,...,...,...,...
246,244,0.080569,MULTIPOLYGON (((-73.94068822000003 40.85131543...,0.000360,Washington Heights South,244,Manhattan
248,246,0.069467,MULTIPOLYGON (((-74.00439976203513 40.76267135...,0.000281,West Chelsea/Hudson Yards,246,Manhattan
251,249,0.036384,MULTIPOLYGON (((-74.00250642399995 40.72901638...,0.000072,West Village,249,Manhattan
261,261,0.027120,MULTIPOLYGON (((-74.01332610899988 40.70503078...,0.000034,World Trade Center,261,Manhattan


In [150]:
values_not_in_feature2 = manhattan_zones[~manhattan_zones['LocationID'].isin(df['DOLocationID'])]['LocationID']
values_not_in_feature2

103    103
104    103
105    103
Name: LocationID, dtype: int64

In [172]:
manhattan_zones['LocationID']

3        4
9       24
12      12
13      13
40      41
      ... 
246    244
248    246
251    249
261    261
262    262
Name: LocationID, Length: 69, dtype: int64

In [4]:
columns_to_drop = ['VendorID', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
                  'PULocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
                  'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge',
                  'airport_fee']

# Thanksgiving Day Parade

## event day busyness

In [149]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-11.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-11-22")]
df

,DOLocationID,date,month,time,day_of_the_week
4147083,107,2018-11-22,11,10,Thursday
4147084,162,2018-11-22,11,10,Thursday
4147085,209,2018-11-22,11,10,Thursday
4147086,148,2018-11-22,11,11,Thursday
4147087,186,2018-11-22,11,11,Thursday
...,...,...,...,...,...
8152170,238,2018-11-22,11,22,Thursday
8152175,90,2018-11-22,11,22,Thursday
8152178,74,2018-11-22,11,22,Thursday
8152182,116,2018-11-22,11,22,Thursday


In [160]:
test = df[df["DOLocationID"]==103]
test

,DOLocationID,date,month,time,day_of_the_week


In [99]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [100]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [101]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [102]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,55,0.105769
1,4,1,60,0.115385
2,4,2,23,0.044231
3,4,3,36,0.069231
4,4,4,22,0.042308
...,...,...,...,...
1579,263,19,204,0.392308
1580,263,20,242,0.465385
1581,263,21,248,0.476923
1582,263,22,211,0.405769


## baseline busyness

In [103]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-11.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-11-15")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-11-08")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,140,2018-11-15,11,10,Thursday
1,237,2018-11-15,11,10,Thursday
2,170,2018-11-15,11,10,Thursday
3,164,2018-11-15,11,11,Thursday
4,237,2018-11-15,11,14,Thursday
...,...,...,...,...,...
469298,209,2018-11-08,11,21,Thursday
469299,75,2018-11-08,11,21,Thursday
469300,230,2018-11-08,11,22,Thursday
469301,113,2018-11-08,11,22,Thursday


In [104]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [105]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [106]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,47,0.032844
1,4,1,29,0.020266
2,4,2,21,0.014675
3,4,3,16,0.011181
4,4,4,8,0.005590
...,...,...,...,...
1579,263,19,350,0.244584
1580,263,20,361,0.252271
1581,263,21,373,0.260657
1582,263,22,341,0.238295


In [107]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,47,0.032844,55,0.105769
1,4,1,29,0.020266,60,0.115385
2,4,2,21,0.014675,23,0.044231
3,4,3,16,0.011181,36,0.069231
4,4,4,8,0.005590,22,0.042308
...,...,...,...,...,...,...
1579,263,19,350,0.244584,204,0.392308
1580,263,20,361,0.252271,242,0.465385
1581,263,21,373,0.260657,248,0.476923
1582,263,22,341,0.238295,211,0.405769


In [108]:
df_compare["Event_ID"]=1
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,1,4,0,47,0.032844,55,0.105769
1,1,4,1,29,0.020266,60,0.115385
2,1,4,2,21,0.014675,23,0.044231
3,1,4,3,16,0.011181,36,0.069231
4,1,4,4,8,0.005590,22,0.042308
...,...,...,...,...,...,...,...
1579,1,263,19,350,0.244584,204,0.392308
1580,1,263,20,361,0.252271,242,0.465385
1581,1,263,21,373,0.260657,248,0.476923
1582,1,263,22,341,0.238295,211,0.405769


In [109]:
df_compare.to_json('Summer Project\\EventHistoricalData\\Thanksgiving.json', orient='records')

# Halloween Parade

## event day busyness

In [50]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-10.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-28")]
df

,DOLocationID,date,month,time,day_of_the_week
1963989,237,2018-10-28,10,16,Sunday
7725484,13,2018-10-28,10,0,Sunday
7726733,4,2018-10-28,10,0,Sunday
7726734,148,2018-10-28,10,0,Sunday
7727519,234,2018-10-28,10,0,Sunday
...,...,...,...,...,...
8833042,74,2018-10-28,10,19,Sunday
8833045,42,2018-10-28,10,20,Sunday
8833048,234,2018-10-28,10,20,Sunday
8833051,48,2018-10-28,10,20,Sunday


In [51]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [52]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [53]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [54]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,132,0.169448
1,4,1,134,0.172015
2,4,2,118,0.151476
3,4,3,104,0.133504
4,4,4,66,0.084724
...,...,...,...,...
1579,263,19,324,0.415918
1580,263,20,328,0.421053
1581,263,21,237,0.304236
1582,263,22,180,0.231065


## baseline busyness

In [55]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-10.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-21")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-10-14")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,142,2018-10-21,10,0,Sunday
1,243,2018-10-21,10,0,Sunday
2,79,2018-10-21,10,0,Sunday
3,142,2018-10-21,10,0,Sunday
4,186,2018-10-21,10,0,Sunday
...,...,...,...,...,...
453412,127,2018-10-14,10,22,Sunday
453413,161,2018-10-14,10,22,Sunday
453414,137,2018-10-14,10,22,Sunday
453415,232,2018-10-14,10,23,Sunday


In [56]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [57]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [58]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,102,0.141667
1,4,1,117,0.162500
2,4,2,109,0.151389
3,4,3,81,0.112500
4,4,4,43,0.059722
...,...,...,...,...
1579,263,19,352,0.488889
1580,263,20,286,0.397222
1581,263,21,271,0.376389
1582,263,22,225,0.312500


In [59]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,102,0.141667,132,0.169448
1,4,1,117,0.162500,134,0.172015
2,4,2,109,0.151389,118,0.151476
3,4,3,81,0.112500,104,0.133504
4,4,4,43,0.059722,66,0.084724
...,...,...,...,...,...,...
1579,263,19,352,0.488889,324,0.415918
1580,263,20,286,0.397222,328,0.421053
1581,263,21,271,0.376389,237,0.304236
1582,263,22,225,0.312500,180,0.231065


In [60]:
df_compare["Event_ID"]=2
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,2,4,0,102,0.141667,132,0.169448
1,2,4,1,117,0.162500,134,0.172015
2,2,4,2,109,0.151389,118,0.151476
3,2,4,3,81,0.112500,104,0.133504
4,2,4,4,43,0.059722,66,0.084724
...,...,...,...,...,...,...,...
1579,2,263,19,352,0.488889,324,0.415918
1580,2,263,20,286,0.397222,328,0.421053
1581,2,263,21,271,0.376389,237,0.304236
1582,2,263,22,225,0.312500,180,0.231065


In [61]:
df_compare.to_json('Summer Project\\EventHistoricalData\\HalloweenParade.json', orient='records')

# Saint Patrick's Day Parade

## event day busyness

In [62]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-03.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-03-17")]
df

,DOLocationID,date,month,time,day_of_the_week
5013309,140,2018-03-17,3,0,Saturday
5014092,249,2018-03-17,3,0,Saturday
5014093,244,2018-03-17,3,0,Saturday
5014108,151,2018-03-17,3,0,Saturday
5015217,151,2018-03-17,3,0,Saturday
...,...,...,...,...,...
5611337,41,2018-03-17,3,18,Saturday
5622584,68,2018-03-17,3,18,Saturday
5630765,234,2018-03-17,3,23,Saturday
5634406,144,2018-03-17,3,23,Saturday


In [63]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [64]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [65]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [66]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,131,0.128180
1,4,1,131,0.128180
2,4,2,86,0.084149
3,4,3,78,0.076321
4,4,4,73,0.071429
...,...,...,...,...
1579,263,19,515,0.503914
1580,263,20,461,0.451076
1581,263,21,459,0.449119
1582,263,22,465,0.454990


## baseline busyness

In [67]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-03.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-03-10")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-03-03")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,140,2018-03-10,3,0,Saturday
1,143,2018-03-10,3,23,Saturday
2,263,2018-03-10,3,0,Saturday
3,262,2018-03-10,3,0,Saturday
4,229,2018-03-10,3,0,Saturday
...,...,...,...,...,...
584147,162,2018-03-03,3,16,Saturday
584148,90,2018-03-03,3,20,Saturday
584149,79,2018-03-03,3,21,Saturday
584150,164,2018-03-03,3,22,Saturday


In [68]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [69]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [70]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,136,0.120567
1,4,1,111,0.098404
2,4,2,102,0.090426
3,4,3,78,0.069149
4,4,4,57,0.050532
...,...,...,...,...
1579,263,19,446,0.395390
1580,263,20,383,0.339539
1581,263,21,362,0.320922
1582,263,22,415,0.367908


In [71]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,136,0.120567,131,0.128180
1,4,1,111,0.098404,131,0.128180
2,4,2,102,0.090426,86,0.084149
3,4,3,78,0.069149,78,0.076321
4,4,4,57,0.050532,73,0.071429
...,...,...,...,...,...,...
1579,263,19,446,0.395390,515,0.503914
1580,263,20,383,0.339539,461,0.451076
1581,263,21,362,0.320922,459,0.449119
1582,263,22,415,0.367908,465,0.454990


In [72]:
df_compare["Event_ID"]=3
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,3,4,0,136,0.120567,131,0.128180
1,3,4,1,111,0.098404,131,0.128180
2,3,4,2,102,0.090426,86,0.084149
3,3,4,3,78,0.069149,78,0.076321
4,3,4,4,57,0.050532,73,0.071429
...,...,...,...,...,...,...,...
1579,3,263,19,446,0.395390,515,0.503914
1580,3,263,20,383,0.339539,461,0.451076
1581,3,263,21,362,0.320922,459,0.449119
1582,3,263,22,415,0.367908,465,0.454990


In [73]:
df_compare.to_json('Summer Project\\EventHistoricalData\\SaintPatricks.json', orient='records')

# Pride March

## event day busyness

In [74]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-06.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-06-24")]
df

,DOLocationID,date,month,time,day_of_the_week
6496829,239,2018-06-24,6,0,Sunday
6519484,107,2018-06-24,6,2,Sunday
6519485,140,2018-06-24,6,2,Sunday
6527202,161,2018-06-24,6,3,Sunday
6598864,90,2018-06-24,6,12,Sunday
...,...,...,...,...,...
7015439,68,2018-06-24,6,20,Sunday
7016321,41,2018-06-24,6,22,Sunday
7016322,236,2018-06-24,6,22,Sunday
7017463,244,2018-06-24,6,20,Sunday


In [75]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [76]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [77]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [78]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,115,0.152926
1,4,1,95,0.126330
2,4,2,107,0.142287
3,4,3,73,0.097074
4,4,4,56,0.074468
...,...,...,...,...
1579,263,19,330,0.438830
1580,263,20,289,0.384309
1581,263,21,271,0.360372
1582,263,22,235,0.312500


## baseline busyness

In [79]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-06.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-06-17")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-06-10")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,68,2018-06-17,6,4,Sunday
1,141,2018-06-17,6,9,Sunday
2,170,2018-06-17,6,9,Sunday
3,74,2018-06-17,6,14,Sunday
4,161,2018-06-17,6,9,Sunday
...,...,...,...,...,...
424139,68,2018-06-10,6,16,Sunday
424140,229,2018-06-10,6,17,Sunday
424141,164,2018-06-10,6,16,Sunday
424142,158,2018-06-10,6,16,Sunday


In [80]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [81]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [82]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,108,0.162162
1,4,1,106,0.159159
2,4,2,82,0.123123
3,4,3,63,0.094595
4,4,4,36,0.054054
...,...,...,...,...
1579,263,19,309,0.463964
1580,263,20,269,0.403904
1581,263,21,273,0.409910
1582,263,22,218,0.327327


In [83]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,108,0.162162,115,0.152926
1,4,1,106,0.159159,95,0.126330
2,4,2,82,0.123123,107,0.142287
3,4,3,63,0.094595,73,0.097074
4,4,4,36,0.054054,56,0.074468
...,...,...,...,...,...,...
1579,263,19,309,0.463964,330,0.438830
1580,263,20,269,0.403904,289,0.384309
1581,263,21,273,0.409910,271,0.360372
1582,263,22,218,0.327327,235,0.312500


In [84]:
df_compare["Event_ID"]=4
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,4,0,108,0.162162,115,0.152926
1,4,4,1,106,0.159159,95,0.126330
2,4,4,2,82,0.123123,107,0.142287
3,4,4,3,63,0.094595,73,0.097074
4,4,4,4,36,0.054054,56,0.074468
...,...,...,...,...,...,...,...
1579,4,263,19,309,0.463964,330,0.438830
1580,4,263,20,269,0.403904,289,0.384309
1581,4,263,21,273,0.409910,271,0.360372
1582,4,263,22,218,0.327327,235,0.312500


In [85]:
df_compare.to_json('Summer Project\\EventHistoricalData\\PrideMarch.json', orient='records')

# TimesSquare Ball

## event day busyness

In [86]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-12.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-12-31")]
df

,DOLocationID,date,month,time,day_of_the_week
4098378,234,2018-12-31,12,16,Monday
7956719,75,2018-12-31,12,0,Monday
7957133,88,2018-12-31,12,0,Monday
7957319,229,2018-12-31,12,0,Monday
7957320,141,2018-12-31,12,0,Monday
...,...,...,...,...,...
8195636,231,2018-12-31,12,22,Monday
8195657,100,2018-12-31,12,22,Monday
8195664,244,2018-12-31,12,23,Monday
8195669,107,2018-12-31,12,23,Monday


In [87]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [88]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [89]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [90]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,47,0.068915
1,4,1,37,0.054252
2,4,2,20,0.029326
3,4,3,15,0.021994
4,4,4,13,0.019062
...,...,...,...,...
1579,263,19,434,0.636364
1580,263,20,358,0.524927
1581,263,21,393,0.576246
1582,263,22,278,0.407625


## baseline busyness

In [91]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-12.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-12-24")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-12-17")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,162,2018-12-24,12,17,Monday
1,79,2018-12-24,12,17,Monday
2,79,2018-12-24,12,17,Monday
3,249,2018-12-24,12,17,Monday
4,230,2018-12-24,12,17,Monday
...,...,...,...,...,...
408869,42,2018-12-17,12,20,Monday
408870,232,2018-12-17,12,20,Monday
408871,90,2018-12-17,12,21,Monday
408872,232,2018-12-17,12,21,Monday


In [92]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [93]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [94]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,32,0.034372
1,4,1,27,0.029001
2,4,2,17,0.018260
3,4,3,8,0.008593
4,4,4,8,0.008593
...,...,...,...,...
1579,263,19,350,0.375940
1580,263,20,343,0.368421
1581,263,21,285,0.306122
1582,263,22,263,0.282492


In [95]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,32,0.034372,47,0.068915
1,4,1,27,0.029001,37,0.054252
2,4,2,17,0.018260,20,0.029326
3,4,3,8,0.008593,15,0.021994
4,4,4,8,0.008593,13,0.019062
...,...,...,...,...,...,...
1579,263,19,350,0.375940,434,0.636364
1580,263,20,343,0.368421,358,0.524927
1581,263,21,285,0.306122,393,0.576246
1582,263,22,263,0.282492,278,0.407625


In [96]:
df_compare["Event_ID"]=5
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,5,4,0,32,0.034372,47,0.068915
1,5,4,1,27,0.029001,37,0.054252
2,5,4,2,17,0.018260,20,0.029326
3,5,4,3,8,0.008593,15,0.021994
4,5,4,4,8,0.008593,13,0.019062
...,...,...,...,...,...,...,...
1579,5,263,19,350,0.375940,434,0.636364
1580,5,263,20,343,0.368421,358,0.524927
1581,5,263,21,285,0.306122,393,0.576246
1582,5,263,22,263,0.282492,278,0.407625


In [97]:
df_compare.to_json('Summer Project\\EventHistoricalData\\TimesSquareBall.json', orient='records')

# Lunar New Year Parade

## event day busyness

In [110]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-02.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-02-25")]
df

,DOLocationID,date,month,time,day_of_the_week
7307338,87,2018-02-25,2,0,Sunday
7310427,237,2018-02-25,2,0,Sunday
7310835,90,2018-02-25,2,0,Sunday
7310923,41,2018-02-25,2,0,Sunday
7310969,79,2018-02-25,2,0,Sunday
...,...,...,...,...,...
7601755,144,2018-02-25,2,23,Sunday
7601761,163,2018-02-25,2,23,Sunday
7602418,158,2018-02-25,2,20,Sunday
7603926,230,2018-02-25,2,23,Sunday


In [111]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [112]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [113]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [114]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,143,0.156627
1,4,1,137,0.150055
2,4,2,116,0.127054
3,4,3,110,0.120482
4,4,4,86,0.094195
...,...,...,...,...
1579,263,19,331,0.362541
1580,263,20,266,0.291347
1581,263,21,173,0.189485
1582,263,22,189,0.207010


## baseline busyness

In [115]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-02.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-02-18")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-02-11")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,142,2018-02-18,2,0,Sunday
1,13,2018-02-18,2,23,Sunday
2,151,2018-02-18,2,0,Sunday
3,158,2018-02-18,2,0,Sunday
4,158,2018-02-18,2,0,Sunday
...,...,...,...,...,...
451509,163,2018-02-11,2,23,Sunday
451510,239,2018-02-11,2,23,Sunday
451511,161,2018-02-11,2,23,Sunday
451512,243,2018-02-11,2,23,Sunday


In [116]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [117]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [118]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,133,0.156287
1,4,1,125,0.146886
2,4,2,110,0.129260
3,4,3,85,0.099882
4,4,4,50,0.058754
...,...,...,...,...
1579,263,19,271,0.318449
1580,263,20,235,0.276146
1581,263,21,236,0.277321
1582,263,22,182,0.213866


In [119]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,133,0.156287,143,0.156627
1,4,1,125,0.146886,137,0.150055
2,4,2,110,0.129260,116,0.127054
3,4,3,85,0.099882,110,0.120482
4,4,4,50,0.058754,86,0.094195
...,...,...,...,...,...,...
1579,263,19,271,0.318449,331,0.362541
1580,263,20,235,0.276146,266,0.291347
1581,263,21,236,0.277321,173,0.189485
1582,263,22,182,0.213866,189,0.207010


In [120]:
df_compare["Event_ID"]=6
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,6,4,0,133,0.156287,143,0.156627
1,6,4,1,125,0.146886,137,0.150055
2,6,4,2,110,0.129260,116,0.127054
3,6,4,3,85,0.099882,110,0.120482
4,6,4,4,50,0.058754,86,0.094195
...,...,...,...,...,...,...,...
1579,6,263,19,271,0.318449,331,0.362541
1580,6,263,20,235,0.276146,266,0.291347
1581,6,263,21,236,0.277321,173,0.189485
1582,6,263,22,182,0.213866,189,0.207010


In [121]:
df_compare.to_json('Summer Project\\EventHistoricalData\\LunarNewYear.json', orient='records')

# Three Kings Day Parade

## event day busyness

In [122]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-01.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-01-05")]
df

,DOLocationID,date,month,time,day_of_the_week
859831,224,2018-01-05,1,0,Friday
859885,50,2018-01-05,1,0,Friday
860029,161,2018-01-05,1,0,Friday
860303,4,2018-01-05,1,0,Friday
860778,158,2018-01-05,1,0,Friday
...,...,...,...,...,...
1156426,107,2018-01-05,1,22,Friday
1159228,13,2018-01-05,1,23,Friday
1160958,161,2018-01-05,1,7,Friday
1160959,90,2018-01-05,1,7,Friday


In [123]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [124]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [125]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [126]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,28,0.024911
1,4,1,18,0.016014
2,4,2,11,0.009786
3,4,3,11,0.009786
4,4,4,10,0.008897
...,...,...,...,...
1579,263,19,505,0.449288
1580,263,20,430,0.382562
1581,263,21,328,0.291815
1582,263,22,388,0.345196


## baseline busyness

In [127]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-01.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-01-12")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-01-19")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,161,2018-01-12,1,6,Friday
1,162,2018-01-12,1,6,Friday
2,162,2018-01-12,1,6,Friday
3,229,2018-01-12,1,6,Friday
4,142,2018-01-12,1,6,Friday
...,...,...,...,...,...
572367,162,2018-01-19,1,22,Friday
572368,42,2018-01-19,1,17,Friday
572369,116,2018-01-19,1,18,Friday
572370,41,2018-01-19,1,19,Friday


In [128]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [129]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [130]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,70,0.049505
1,4,1,48,0.033946
2,4,2,30,0.021216
3,4,3,24,0.016973
4,4,4,16,0.011315
...,...,...,...,...
1579,263,19,545,0.385431
1580,263,20,472,0.333805
1581,263,21,400,0.282885
1582,263,22,383,0.270863


In [131]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,70,0.049505,28,0.024911
1,4,1,48,0.033946,18,0.016014
2,4,2,30,0.021216,11,0.009786
3,4,3,24,0.016973,11,0.009786
4,4,4,16,0.011315,10,0.008897
...,...,...,...,...,...,...
1579,263,19,545,0.385431,505,0.449288
1580,263,20,472,0.333805,430,0.382562
1581,263,21,400,0.282885,328,0.291815
1582,263,22,383,0.270863,388,0.345196


In [132]:
df_compare["Event_ID"]=7
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,7,4,0,70,0.049505,28,0.024911
1,7,4,1,48,0.033946,18,0.016014
2,7,4,2,30,0.021216,11,0.009786
3,7,4,3,24,0.016973,11,0.009786
4,7,4,4,16,0.011315,10,0.008897
...,...,...,...,...,...,...,...
1579,7,263,19,545,0.385431,505,0.449288
1580,7,263,20,472,0.333805,430,0.382562
1581,7,263,21,400,0.282885,328,0.291815
1582,7,263,22,383,0.270863,388,0.345196


In [133]:
df_compare.to_json('Summer Project\\EventHistoricalData\\ThreeKingsDay.json', orient='records')

# Macy 4th of July Fireworks

## event day busyness

In [161]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-07.parquet")
df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-07-04")]
df

,DOLocationID,date,month,time,day_of_the_week
714412,163,2018-07-04,7,0,Wednesday
714461,113,2018-07-04,7,0,Wednesday
714789,163,2018-07-04,7,0,Wednesday
715120,13,2018-07-04,7,0,Wednesday
715311,141,2018-07-04,7,0,Wednesday
...,...,...,...,...,...
7850613,79,2018-07-04,7,0,Wednesday
7850615,262,2018-07-04,7,12,Wednesday
7850616,74,2018-07-04,7,13,Wednesday
7850618,233,2018-07-04,7,18,Wednesday


In [135]:
# Create an aggregated DataFrame
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [136]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [137]:
df_event = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_event= pd.concat([df_event,df],axis=0)

In [138]:
df_event.reset_index(drop=True, inplace=True)
df_event = df_event.rename(columns={'num_dropoffs':'num_dropoffs_event'})
scaler = MinMaxScaler()
df_event['busyness_score_event'] = scaler.fit_transform(df_event[['num_dropoffs_event']])
df_event

,DOLocationID,time,num_dropoffs_event,busyness_score_event
0,4,0,76,0.124386
1,4,1,53,0.086743
2,4,2,41,0.067103
3,4,3,35,0.057283
4,4,4,25,0.040917
...,...,...,...,...
1579,263,19,229,0.374795
1580,263,20,187,0.306056
1581,263,21,165,0.270049
1582,263,22,306,0.500818


## baseline busyness

In [139]:
df = pd.read_parquet("Summer Project\\taxi\\yellow_tripdata_2018-07.parquet")

df['date'] = df['tpep_pickup_datetime'].dt.normalize()
df['month'] = df['tpep_pickup_datetime'].dt.month
df['time'] = df['tpep_pickup_datetime'].dt.hour
df['day_of_the_week'] = df['date'].dt.day_name()
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

df = df.drop(columns=columns_to_drop)
df = df.dropna()
#     df = df[df['PULocationID'].isin(manhattan_zones['LocationID'])]
df = df[df['DOLocationID'].isin(manhattan_zones['LocationID'])]
df1 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-07-11")]
df2 = df[(df['time'] >= 0) & (df['time'] <= 23)&(df['date']=="2018-07-18")]
df = pd.concat([df1, df2], ignore_index=True)
df

,DOLocationID,date,month,time,day_of_the_week
0,79,2018-07-11,7,3,Wednesday
1,88,2018-07-11,7,5,Wednesday
2,13,2018-07-11,7,5,Wednesday
3,163,2018-07-11,7,6,Wednesday
4,163,2018-07-11,7,6,Wednesday
...,...,...,...,...,...
513349,234,2018-07-18,7,22,Wednesday
513350,161,2018-07-18,7,21,Wednesday
513351,186,2018-07-18,7,22,Wednesday
513352,141,2018-07-18,7,23,Wednesday


In [140]:
df_agg = df.groupby(['DOLocationID', 'time']).size().reset_index(name='num_dropoffs')

separated_lists = {}

# Iterate over the data list
for index, row in df_agg.iterrows():
    category = row["DOLocationID"]
    
    # Check if the category is already a key in the dictionary
    if category in separated_lists:
        separated_lists[category].append(row)
    else:
        separated_lists[category] = [row]

dfs = {}
for category, rows in separated_lists.items():
    # Convert the list of rows to a DataFrame
    df = pd.DataFrame(rows)
    
    # Store the DataFrame in the dictionary with the category as the key
    dfs[category] = df

In [141]:
# Iterate over the categories in dfs
for category, df in dfs.items():
    # Create a set of existing time values in the current category
    existing_times = set(df['time'])
    
    # Check if any time value is missing from 0 to 23
    missing_times = set(range(24)) - existing_times
    
    # Add missing entries with other features set as "null"
    if missing_times:
        missing_data = [{'time': time, 'DOLocationID': category,"num_dropoffs":0} for time in missing_times]
        missing_df = pd.DataFrame(missing_data)
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values('time')
        dfs[category] = df

In [142]:
df_bl = pd.DataFrame()
for category, df in dfs.items():
    df = pd.DataFrame(df)
    df_bl= pd.concat([df_bl,df],axis=0)
df_bl.reset_index(inplace=True)
df_bl=df_bl.drop("index",axis=1)
df_bl["num_dropoffs"] =df_bl["num_dropoffs"] //2
df_bl['busyness_score'] = scaler.fit_transform(df_bl[['num_dropoffs']])
df_bl

,DOLocationID,time,num_dropoffs,busyness_score
0,4,0,46,0.030605
1,4,1,26,0.017299
2,4,2,22,0.014637
3,4,3,15,0.009980
4,4,4,3,0.001996
...,...,...,...,...
1579,263,19,458,0.304724
1580,263,20,423,0.281437
1581,263,21,455,0.302728
1582,263,22,407,0.270792


In [143]:
df_compare= pd.concat([df_bl,df_event["num_dropoffs_event"],df_event["busyness_score_event"]],axis=1)
df_compare

,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,4,0,46,0.030605,76,0.124386
1,4,1,26,0.017299,53,0.086743
2,4,2,22,0.014637,41,0.067103
3,4,3,15,0.009980,35,0.057283
4,4,4,3,0.001996,25,0.040917
...,...,...,...,...,...,...
1579,263,19,458,0.304724,229,0.374795
1580,263,20,423,0.281437,187,0.306056
1581,263,21,455,0.302728,165,0.270049
1582,263,22,407,0.270792,306,0.500818


In [144]:
df_compare["Event_ID"]=8
df_compare=df_compare[["Event_ID","DOLocationID","time","num_dropoffs","busyness_score","num_dropoffs_event","busyness_score_event"]]
df_compare.reset_index(drop=True, inplace=True)
df_compare

,Event_ID,DOLocationID,time,num_dropoffs,busyness_score,num_dropoffs_event,busyness_score_event
0,8,4,0,46,0.030605,76,0.124386
1,8,4,1,26,0.017299,53,0.086743
2,8,4,2,22,0.014637,41,0.067103
3,8,4,3,15,0.009980,35,0.057283
4,8,4,4,3,0.001996,25,0.040917
...,...,...,...,...,...,...,...
1579,8,263,19,458,0.304724,229,0.374795
1580,8,263,20,423,0.281437,187,0.306056
1581,8,263,21,455,0.302728,165,0.270049
1582,8,263,22,407,0.270792,306,0.500818


In [145]:
df_compare.to_json('Summer Project\\EventHistoricalData\\4thOfJulyFireworks.json', orient='records')

In [166]:
check = pd.read_csv("2022_unique_zones.csv")
check

,LocationID,Borough,Zone,service_zone
0,4,Manhattan,Alphabet City,Yellow Zone
1,12,Manhattan,Battery Park,Yellow Zone
2,13,Manhattan,Battery Park City,Yellow Zone
3,24,Manhattan,Bloomingdale,Yellow Zone
4,41,Manhattan,Central Harlem,Boro Zone
...,...,...,...,...
62,246,Manhattan,West Chelsea/Hudson Yards,Yellow Zone
63,249,Manhattan,West Village,Yellow Zone
64,261,Manhattan,World Trade Center,Yellow Zone
65,262,Manhattan,Yorkville East,Yellow Zone


In [167]:
values_not_in_feature2 = check[~check['LocationID'].isin(df['DOLocationID'])]['LocationID']
values_not_in_feature2

18    105
Name: LocationID, dtype: int64

In [168]:
a = sorted(df['DOLocationID'].unique())
a

[4,
 12,
 13,
 24,
 41,
 42,
 43,
 45,
 48,
 50,
 68,
 74,
 75,
 79,
 87,
 88,
 90,
 100,
 107,
 113,
 114,
 116,
 120,
 125,
 127,
 128,
 137,
 140,
 141,
 142,
 143,
 144,
 148,
 151,
 152,
 153,
 158,
 161,
 162,
 163,
 164,
 166,
 170,
 186,
 194,
 202,
 209,
 211,
 224,
 229,
 230,
 231,
 232,
 233,
 234,
 236,
 237,
 238,
 239,
 243,
 244,
 246,
 249,
 261,
 262,
 263]